MNIST 데이터셋을 위한 텐서플로 실습파일입니다.

해당 실습 문서는 TensorFlow 공식 Tutorial을 기반으로 수정하였습니다.



현재 단계에서 아래 코드에 대해 한 줄 한 줄 이해하실 필요는 없습니다.

이후 수업이 진행됨에 따라 코드를 더 잘 이해할 수 있게 될테니 

우선은 실행을 해서 학습이 제대로 돌아가는지를 보며 결과를 확인하는 것을 목적으로 하겠습니다. 
 


먼저 프로그램에 텐서플로 라이브러리를 임포트합니다

In [ ]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential

다음으로 MNIST 데이터셋을 로드하여 준비합니다.

MNIST는 굉장히 유명한 딥러닝 입문용 데이터이기 때문에, 바로 다운로드 받을 수 있도록 텐서플로에서 관련 기능을 제공하고 있습니다.

일단은 아래 내용을 실행합시다.

In [ ]:
# data load
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data() 

# 0~255의 픽셀 값을 0~1로 조정
x_train, x_test = x_train / 255.0, x_test / 255.0

# shape 조정
x_train = x_train.reshape(60000,28,28,1)
x_test = x_test.reshape(10000,28,28,1)

# 정답 label을 길이10 벡터화
y_train = tf.one_hot(y_train, depth=10)
y_test = tf.one_hot(y_test, depth=10)

# Training set으로부터 Validation set을 따로 분리.
x_train, x_val = x_train[:50000], x_train[50000:]
y_train, y_val = y_train[:50000], y_train[50000:]

데이터셋의 shape를 확인해보겠습니다. 

shape은 각 데이터(텐서)가 어떻게 생겼는지 모양을 표현하는 방식이라고 설명드렸었습니다.

입력 데이터의 경우, (28,28,1) 짜리의 이미지가 학습용으로 5만 장, 검증용으로 1만 장, 테스트용으로 1만 장, 총 7만 장의 이미지가 있음을 확인할 수 있습니다.

정답 데이터(라벨)의 경우 길이 10짜리의 벡터가 학습용으로 역시 5만 개, 검증용 1만 개, 테스트용 1만 개 총 7만 개가 있음을 확인할 수 있습니다.

In [ ]:
print("Train set input shape : \t", x_train.shape)  # 학습용 이미지의 shape
print("Validation set input shape : \t", x_val.shape)  # 검증용 이미지의 shape
print("Test set input shape : \t\t", x_test.shape)  # 테스트용 이미지의 shape
print()
print("Train set output shape : \t", y_train.shape)  # 학습용 라벨(정답)의 shape
print("Validation set output shape : \t", y_val.shape)  # 검증용 라벨(정답)의 shape
print("Test set output shape : \t", y_test.shape)  # 테스트용 라벨(정답)의 shape

해당하는 Dataset의 shape와 이미지는 아래와 같습니다.

아래 코드에서 index 부분에 다양한 값을 입력하여 실행해보세요 (0~49999 내의 값)

1001번 INDEX를 입력하여 이미지를 확인해보는 것을 권장드립니다.

> INDEX = 1001

이 이미지는 4인지 7인지 애매합니다. 

하지만 label을 확인해보니 숫자 7을 손으로 쓴 글씨군요.

우리가 만든 모델이 이렇게 애매한 이미지도 잘 인식할 수 있을지 궁금해지네요 🙄



In [ ]:
import matplotlib.pyplot as plt

INDEX = 0

plt.imshow(x_train[INDEX].reshape(28,28), cmap='gray_r')

print("입력 이미지의 shape : ", x_train[INDEX].shape)
print("해당 데이터에 대한 label : ", y_train[INDEX])

인공신경망 아키텍처를 구성합니다. (교재상의 모델 구현)

아키텍쳐가 기억나지 않는 분은 교재 1-5 섹션으로 돌아가 그림을 확인합시다.

Sequential() 내부에 모델의 레이어를 하나씩 추가하여 아키텍처를 구성할 수 있습니다.

* Flatten() 레이어는 28*28 픽셀의 이미지를 차원을 무시하고, 납작하게 눌러서(flatten) 1차원의 열벡터로 펼치는 역할을 합니다.
* Dense 레이어는 FNN의 한 층을 구성합니다. 첫 번째 인자가 해당 레이어의 뉴런(노드) 수를 의미합니다. activation 옵션으로는 각 뉴런이 가중합을 하고 난 후 적용할 비선형 함수를 지정할 수 있습니다. 여기에는 우리가 배웠던 시그모이드 함수(S자 모양의, 0~1로 변환해주는 함수)명을 입력하겠습니다.
* 마지막 Dense 레이어는 output으로 출력할 뉴런 수 (10개의 숫자로 분류해야 하므로 10)를 지정하고 activation에는 'softmax'를 설정합니다. 


In [ ]:
model = Sequential([       ###################### MISSION 정답 확인 ########################
                    Flatten(input_shape=(28,28,1)),
                    Dense(50, activation='sigmoid'),
                    Dense(40, activation='sigmoid'),
                    Dense(30, activation='sigmoid'),
                    Dense(10, activation='softmax')
])

model.summary() 기능을 활용하여 위에서 만들어놓았던 Model의 아키텍처와 학습할 Parameter 수를 확인할 수 있습니다.

첫 번째 Flatten layer에서는 output shape이 (None, 784) 입니다.

앞으로 데이터가 몇 건 들어올지는 몰라도, 한 건당 784개의 값이 존재한다는 뜻입니다.

두 번째 Dense 레이어에서는 output shape이 (None, 15)인데, 역시 데이터가 몇 건인지는 모르겠지만 한 건당 15개의 값이 될거라는 뜻입니다. 두 번째 레이어의 노드 수를 15개로 설정했기 때문입니다.

마찬가지로 세 번째 레이어의 shape은 (None, 10)이 됩니다.

In [ ]:
model.summary()

훈련에 필요한 파라미터 업데이트 기법과 loss 함수를 선택합니다.

파라미터 업데이트 방식은 **SGD(Stochastic Gradient Descent, 경사하강법)** 입니다.

파라미터 업데이트는 파라미터를 최적화한다는 의미에서 **optimzer**라고도 합니다.

Loss 함수로는 교재에서 언급한 **MSE(Mean Squared Error) Loss** 함수를 이용하겠습니다. 

성능 평가는 **분류정확도(accuracy)**를 기준으로 합니다.

분류 정확도는 데이터를 몇 개나 올바로 분류했는지를 비율로 나타냅니다. 

분류 정확도가 1에 가까울수록 좋겠지요?


SGD, MSE, Accuracy 모두 계산 수식이 있지만, 수식을 몰라도 이름만 알고 있다면 코드 상에서 간편하게 적용해볼수 있습니다.

In [ ]:
my_opt = tf.keras.optimizers.SGD(learning_rate=10.0)  ###################### MISSION 정답 확인 ########################
my_loss = tf.keras.losses.MeanSquaredError()
model.compile(optimizer=my_opt, loss=my_loss, metrics=['acc'])

이제 모델을 5 Epoch 동안 학습하면서, 테스트할 예정입니다.

학습 데이터를 5번 반복해서 정답을 알려주면서 이 정답을 더 잘 맞추도록 모델을 최적화하겠다는 의미입니다.

학습이 진행되는 동안 측정 지표(accuracy)를 출력합니다.

mini batch 크기는 32로 설정합니다. 

한 번에 MNIST 데이터 모두를 보여주어 업데이트하지 않고, 32건씩 할부로 입력하고 학습하겠다는 의미입니다.

model.fit() 기능을 활용하여 모델의 학습(loss를 줄이는 방향으로 parameter 업데이트)을 수행합니다.

다섯 번의 epoch이 진행되는 동안 출력 로그가 잘 찍히는지 봅시다.

In [ ]:
EPOCHS = 50  ###################### MISSION 정답 확인 ########################
BATCH = 32
history = model.fit(x=x_train, y=y_train, batch_size=BATCH, epochs=EPOCHS, validation_data=(x_val, y_val), verbose=1)

학습용 데이터로 모델 학습이 완료되었습니다.

별도로 떼어놓았던 테스트 데이터를 모델에 입력하고, 추론 결과를 꺼내 테스트셋 성능을 체크합니다

model.evaluate() 기능을 활용하여 모델의 추론 결과를 확인할 수 있습니다.

In [ ]:
model.evaluate(x_test, y_test)

학습이 진행되는 동안 학습 데이터에 대한 Accuracy, Loss를 그래프로 그려볼 수 있습니다. 



In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(5,5))
plt.plot(range(1,EPOCHS+1), history.history['acc'], label="Train")
plt.plot(range(1,EPOCHS+1), history.history['val_acc'], label="Validation")

plt.title("Accuracy Graph")
plt.xticks(range(1,EPOCHS+1), range(1,EPOCHS+1))
plt.legend(loc=2)

plt.show()

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(range(1,EPOCHS+1), history.history['loss'], label="Train")
plt.plot(range(1,EPOCHS+1), history.history['val_loss'], label="Validation")

plt.title("Loss Graph")
plt.xticks(range(1,EPOCHS+1), range(1,EPOCHS+1))
plt.legend(loc=3)

plt.show()

5번의 반복학습동안 Accuracy는 점점 증가하고 Loss는 점점 떨어지고 있군요!

제대로 학습이 잘 진행되는 것 같습니다. 

혹시 우상향하는 Acc 그래프와 우하향하는 Loss그래프가 나오지 않은 분은 코드를 다시 처음부터 차례대로 실행해보시고, 

제공된 코드에서 내가 무언가 잘못 수정한 내용이 없는지 확인해보세요.

**👋잠깐!**

교재로 돌아가서 **'1-9 하이퍼파라미터'** 섹션을 수강한 뒤 실습 미션을 수행해보세요

# 실습 MISSION #1 : 하이퍼파라미터 조정하기

* 첫 번째 레이어 node(뉴런) 수 조정하기 : node 수 50 
* 두 번째 Dense 레이어를 추가 : node 수 40
* 세 번째 Dense 레이어를 추가 : node 수 30
* Learning Rate 값 조절 : 10.0
* 50 epoch동안 돌려보고 테스트 정확도를 확인해봅니다.

(힌트:learning rate은 파라미터를 업데이트할 때 이용됩니다. 그렇다면 어떤 부분에 넣어야 할까요?)

colab에서 특정 기능 위 커서를 올려놓고 대기하면 기능에 대한 설명이 오버랩됩니다. 

이를 잘 활용하여 미션을 수행해봅시다 😊



# 실습 MISSION #2

더 나은 성능을 내기 위해 네트워크를 수정해보자!

아래 항목을 자유로이 변경하고 **test set**에 대한 성능을 확인해봅시다

---



* 레이어 수 
* 레이어 당 노드 수
* learning rate
* epoch 수
* mini-batch size
